In [39]:
#Import Libraries

import sqlite3
import numpy as np 
import pandas as pd 
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [9]:
#Load data into DataFrames

df_sales = pd.read_csv('../data/EXTR_RPSale.csv')
df_res = pd.read_csv('../data/EXTR_ResBldg.csv')
df_lookup = pd.read_csv('../data/EXTR_LookUp.csv')
df_parcel = pd.read_csv('../data/EXTR_Parcel.csv', encoding='latin-1')

In [10]:
#Create 'Year' and 'MajorMinor' columns in df_sales

df_sales['year'] = pd.DatetimeIndex(df_sales['DocumentDate']).year
df_sales['Major'] = df_sales['Major'].astype(str)
df_sales['Minor'] = df_sales['Minor'].astype(str)
df_sales['MajorMinor'] = df_sales['Major'] + '-' + df_sales['Minor']

In [11]:
#Filter for 2019 

df_sales_19 = df_sales[df_sales['year']==2019]

In [12]:
#Filter for PropertyType and SalePrice 

q1 = ("""SELECT * FROM df_sales_19
WHERE (PropertyType = 11 or PropertyType = 12 or PropertyType = 13 or PropertyType = 14) and SalePrice > 0
""")

In [13]:
df_sales_19 = ps.sqldf(q1)

In [14]:
#Create'MajorMinor' columns in df_res

df_res['Major'] = df_res['Major'].astype(str)
df_res['Minor'] = df_res['Minor'].astype(str)
df_res['MajorMinor'] = df_res['Major'] + '-' + df_res['Minor']

In [15]:
#Create'MajorMinor' columns in df_parcel

df_parcel['Major'] = df_parcel['Major'].astype(str)
df_parcel['Minor'] = df_parcel['Minor'].astype(str)
df_parcel['MajorMinor'] = df_parcel['Major'] + '-' + df_parcel['Minor']

In [16]:
#Merge Sales and Parcel 

new_df = pd.merge(df_sales_19, df_parcel, how = 'left',on =['MajorMinor'])

In [17]:
#Merge the merged DF with Res 

final_df = pd.merge(new_df, df_res, how = 'left',on =['MajorMinor'])

In [18]:
#Create a waterfront column 

final_df['is_waterfront'] = final_df['WfntLocation'] > 0

In [19]:
#How many waterfront in our DF 

final_df['WfntLocation'].value_counts()

0.0    19586
8.0       99
6.0       93
9.0       56
3.0       55
7.0       28
5.0        3
1.0        2
4.0        1
Name: WfntLocation, dtype: int64

In [20]:
final_df.shape

(22931, 158)

In [23]:
final_df.describe()

,ExciseTaxNbr,SalePrice,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,year,Range,Township,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
count,2.293100e+04,2.293100e+04,22931.000000,22931.000000,22931.00000,22931.000000,22931.000000,22931.0,19923.000000,19923.000000,...,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000
mean,3.000581e+06,8.415076e+05,11.338755,5.585278,3.10331,1.002965,7.389124,2019.0,4.588516,23.823571,...,0.303639,0.067406,0.175639,1974.554219,95.131200,0.597677,0.123716,0.006761,3.434942,781.840206
std,1.688221e+04,2.315672e+06,0.933239,1.243877,1.39005,0.224512,1.651478,0.0,1.133155,1.669224,...,0.501299,0.258835,0.399190,30.522600,425.617303,6.852864,2.517477,0.433221,0.679666,2683.674905
min,2.968844e+06,1.000000e+01,11.000000,0.000000,2.00000,1.000000,0.000000,2019.0,2.000000,19.000000,...,0.000000,0.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.986058e+06,4.395000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,4.000000,23.000000,...,0.000000,0.000000,0.000000,1954.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,3.001453e+06,6.300000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,4.000000,24.000000,...,0.000000,0.000000,0.000000,1978.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,3.014598e+06,8.850000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,5.000000,25.000000,...,1.000000,0.000000,0.000000,2001.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
max,3.035196e+06,1.630510e+08,14.000000,7.000000,27.00000,18.000000,9.000000,2019.0,13.000000,26.000000,...,5.000000,2.000000,3.000000,2019.000000,2019.000000,100.000000,99.000000,40.000000,5.000000,60000.000000


In [34]:
final_df['SqFtProp'] = final_df['SqFtTotLiving']/final_df['SqFtLot']

In [35]:
final_df['SqFtProp'].describe()

count    19375.000000
mean         0.345664
std          0.311609
min          0.000565
25%          0.157588
50%          0.254000
75%          0.428726
max          4.509395
Name: SqFtProp, dtype: float64

In [36]:
final_df['CostSqFt'] = final_df['SalePrice']/final_df['SqFtTotLiving']

In [37]:
final_df['CostSqFt'].describe()

count    19375.000000
mean       393.728628
std        364.712540
min          0.002268
25%        258.123872
50%        350.318471
75%        466.145139
max      29629.629630
Name: CostSqFt, dtype: float64

In [79]:
fsm_df_3 = final_df[["SalePrice", "BldgGrade"]].copy()
fsm_df_3.dropna(inplace=True)
fsm_3 = ols(formula="SalePrice ~ BldgGrade", data=fsm_df_3).fit()
fsm_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     9865.
Date:                Tue, 03 Mar 2020   Prob (F-statistic):               0.00
Time:                        11:40:38   Log-Likelihood:            -2.8173e+05
No. Observations:               19375   AIC:                         5.635e+05
Df Residuals:                   19373   BIC:                         5.635e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.512e+06   2.36e+04    -64.186      0.000   -1.56e+06   -1.47e+06
BldgGrade   3.006e+05   3026.963     99.321      0.000    2.95e+05    3.07e+05
==============================================================================
Omnibus:                    28545.481   Durbin-Watson:                   1.787
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         26032063.399
Skew:                           8.655   Prob(JB):                         0.00
Kurtosis:                     181.736   Cond. No.                         51.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
fsm_df = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "CostSqFt"]].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + CostSqFt", data=fsm_df).fit()
fsm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                 1.271e+04
Date:                Tue, 03 Mar 2020   Prob (F-statistic):               0.00
Time:                        11:38:52   Log-Likelihood:            -2.7518e+05
No. Observations:               19375   AIC:                         5.504e+05
Df Residuals:                   19371   BIC:                         5.504e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.812e+05   7492.040    -64.230      0.000   -4.96e+05   -4.67e+05
SqFtTotLiving   409.3854      2.642    154.932      0.000     404.206     414.565
SqFtProp       8.168e+04   8233.152      9.921      0.000    6.55e+04    9.78e+04
CostSqFt        955.7403      7.077    135.050      0.000     941.869     969.612
==============================================================================
Omnibus:                    23472.225   Durbin-Watson:                   2.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         68231497.856
Skew:                           5.405   Prob(JB):                         0.00
Kurtosis:                     293.520   Cond. No.                     8.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514509 entries, 0 to 514508
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               514509 non-null  object 
 1   Minor               514509 non-null  object 
 2   BldgNbr             514509 non-null  int64  
 3   NbrLivingUnits      514509 non-null  int64  
 4   Address             514509 non-null  object 
 5   BuildingNumber      514509 non-null  object 
 6   Fraction            514509 non-null  object 
 7   DirectionPrefix     514011 non-null  object 
 8   StreetName          514509 non-null  object 
 9   StreetType          514509 non-null  object 
 10  DirectionSuffix     514011 non-null  object 
 11  ZipCode             469267 non-null  object 
 12  Stories             514509 non-null  float64
 13  BldgGrade           514509 non-null  int64  
 14  BldgGradeVar        514509 non-null  int64  
 15  SqFt1stFloor        514509 non-nul

In [24]:
df_parcel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614337 entries, 0 to 614336
Data columns (total 82 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Major                   614337 non-null  object 
 1   Minor                   614337 non-null  object 
 2   PropName                601485 non-null  object 
 3   PlatName                520002 non-null  object 
 4   PlatLot                 614337 non-null  object 
 5   PlatBlock               614337 non-null  object 
 6   Range                   614337 non-null  int64  
 7   Township                614337 non-null  int64  
 8   Section                 614337 non-null  int64  
 9   QuarterSection          614337 non-null  object 
 10  PropType                614337 non-null  object 
 11  Area                    614307 non-null  float64
 12  SubArea                 614307 non-null  float64
 13  SpecArea                17333 non-null   float64
 14  SpecSubArea         

In [65]:
final_df['TrafficNoise'].value_counts()

0.0    17160
1.0     1654
2.0      969
3.0      140
Name: TrafficNoise, dtype: int64

In [ ]:
#0 is None
#1 is Moderate
#2 is High
#3 is Extreme 

In [62]:
qt = ("""
SELECT SalePrice,
CASE
    WHEN TrafficNoise > 0 THEN 1
    ELSE 0
END AS traffic
FROM final_df
""")
traffic_df = ps.sqldf(qt)

In [66]:
final_df['Traffic'] = traffic_df['traffic']

In [77]:
traffic_df['traffic'].value_counts()

0    20168
1     2763
Name: traffic, dtype: int64

In [ ]:
fsm_df = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "CostSqFt"]].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + CostSqFt", data=fsm_df).fit()
fsm.summary()

In [75]:
fsm_df_2 = final_df[["SalePrice", "Traffic"]].copy()
fsm_2 = ols(formula="SalePrice ~ Traffic", data=fsm_df_2).fit()
fsm_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.3993
Date:                Tue, 03 Mar 2020   Prob (F-statistic):              0.527
Time:                        11:35:30   Log-Likelihood:            -3.6860e+05
No. Observations:               22931   AIC:                         7.372e+05
Df Residuals:                   22929   BIC:                         7.372e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.379e+05   1.63e+04     51.387      0.000    8.06e+05     8.7e+05
Traffic     2.969e+04    4.7e+04      0.632      0.527   -6.24e+04    1.22e+05
==============================================================================
Omnibus:                    67916.877   Durbin-Watson:                   1.716
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       5027258184.668
Skew:                          42.442   Prob(JB):                         0.00
Kurtosis:                    2295.251   Cond. No.                         3.12
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
final_df['AirportNoise'].value_counts()

0.0     19915
64.0        3
60.0        2
61.0        2
62.0        1
Name: AirportNoise, dtype: int64

In [94]:
final_df['PowerLines'].value_counts()

Series([], Name: PowerLines, dtype: int64)

In [92]:
qpl = ("""
SELECT SalePrice,
CASE
    WHEN PowerLines == 'Y' THEN 1
    ELSE 0
END AS power_lines
FROM final_df
""")
power_lines_df = ps.sqldf(qpl)

In [93]:
power_lines_df['power_lines'].value_counts()

0    22931
Name: power_lines, dtype: int64

In [87]:
final_df['PowerLines'] = power_lines_df['power_lines']

In [88]:
final_df['PowerLines'].value_counts()

Series([], Name: PowerLines, dtype: int64)

In [ ]:
fsm_df_4 = final_df[["SalePrice", "Traffic"]].copy()
fsm_4 = ols(formula="SalePrice ~ Traffic", data=fsm_df_4).fit()
fsm_4.summary()

In [55]:
final_df['OtherNuisances'].value_counts()

N    19301
Y      622
Name: OtherNuisances, dtype: int64